In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo
import astropy.constants as const

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})
sns.set_context("paper",font_scale=1)



### varstro data

In [ ]:
with open("varsFR.txt", "r") as f:
    vars = [a.rstrip().split("\t") for a in f.readlines()]

vardf = pd.DataFrame(vars[2:],columns=vars[0]) 

desigs = [i[:5]+i[10:15] for i in vardf['Name']]
vdf = pd.DataFrame([desigs,vardf['Classification']],index=['Name', "Type_main"]).T

In [ ]:
# get ST magnitudes
m475w = [list(map(float,a.split("/"))) for a in vardf['F475W']]
big_m475 = np.array([np.max(m) for m in m475w])
small_m475 = np.array([np.min(m) for m in m475w])
m814w = [list(map(float,a.split("/"))) for a in vardf['F814W']]
big_m814 = np.array([np.max(m) for m in m814w])
small_m814 = np.array([np.min(m) for m in m814w])

# calculate luminosity
def cal_lum(mag,wl):
    f = 10**((mag+21.1)/-2.5) *u.erg/u.s/u.cm**2/u.angstrom
    lum = (4*np.pi *cosmo.angular_diameter_distance(redshifts)**2 *f *(wl*u.angstrom)).to(u.erg/u.s)
    return lum

redshifts = np.array([float(z.split("^")[0]) if z[-1]=="r" else float(z) for z in vardf['Redshift']])
lum475b = cal_lum(big_m475,4750)
lum475a = cal_lum(small_m475,4750)
lum814b = cal_lum(big_m814,8140)
lum814a = cal_lum(small_m814,8140)

# save to df
vdf['rp'] = np.array(list(map(float,vardf['r _p'])))
vdf['z'] = redshifts

# # checking the first object luminosity with WISE, close?
# obs_flux = 3.80E-2*u.Jy
# obs_hz = 1.36E+13*u.Hz
# lum = (obs_flux*obs_hz*4*np.pi*
#         cosmo.luminosity_distance(0.7)**2).to(u.erg/u.s)
# lum*10, (lum475a+lum475b)[0]*1000

vdf.head()

In [ ]:
def hist_bin(ax,quant,lab,clr):
    """plot histogram and a straight line at peak bin"""
    try:
        count, bin, _ = ax.hist(quant,bins=np.linspace(quant.min(),quant.max(),int(np.sqrt(len(quant)))),
                histtype='step',color=clr,alpha=0.6)
        maxind = np.argmax(count)
        maxloc = (bin[maxind]+bin[maxind+1])/2
        ax.axvline(maxloc,c=clr,label=lab)
    except:
        pass
    
def meas_df(quartiles,m,ax,dualmask):
    """plot sep histograms of luminosity bins based on list of quartiles"""
    mcuts =  np.percentile(m,quartiles)
    colors = sns.color_palette("magma", len(mcuts)+1)
    for i,c in zip(range(len(mcuts)+1),colors):
        if i==0:
            lbl = f'$<$ {mcuts[i]:.2f}'
            hist_bin(ax,vdf[(m < mcuts[i]) & dualmask]['rp'], lbl, c)
        elif i==len(mcuts):
            lbl = f'$>$ {mcuts[i-1]:.2f}'
            hist_bin(ax,vdf[(m > mcuts[i-1]) & dualmask]['rp'], lbl, c)
        else:
            lbl = f'{mcuts[i-1]:.2f} $-$ {mcuts[i]:.2f}'
            hist_bin(ax,vdf[(m < mcuts[i]) & (m > mcuts[i-1]) & dualmask]['rp'], lbl, c)
    ax.legend(fontsize=8,loc='upper left');


In [ ]:
fig,ax = plt.subplots(3,1,gridspec_kw={'height_ratios': [1,1,1.5]},figsize=(4,7))#,dpi=200)

b =  np.array([a[-2:]=="^b" for a in vardf['Classification']]) # chen22, star based on gemini spectra
c = np.array([a[-2:]=="^c" for a in vardf['Classification']]) # chen22, star based on sdss spectra
chen24star = vardf['Classification'].str.contains("star chen24") # chen 24
gross24single =  vardf['Classification'].str.contains("gross24") # chen 24
zmax = redshifts.max()
zmin = 2
zmask = (vdf['z']> zmin) & (vdf['z']<zmax)
dualmask = ~(chen24star & gross24single) & zmask 
#dualmask = ~ vardf['Classification'].str.contains("star")
#dualmask = np.array([True]*len(vdf))

quant = np.log10(lum475b.value)

meas_df([50],quant,ax[0],dualmask)
meas_df([25,75],quant,ax[1],dualmask)
[a.set_xlabel("Sep(kpc)") for a in ax[:-1]]
[a.set_ylabel("Number of dual") for a in ax[:-1]]

zarr = np.linspace(zmin,zmax)
hstRes = ((0.04*3*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1].plot(zarr,hstRes,alpha=0.5,label="0.12''")
gaiaRes = ((0.4*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1].plot(zarr,gaiaRes,alpha=0.5,label="0.4''")
gaiaResUp = ((0.7*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1].plot(zarr,gaiaResUp,alpha=0.5,label="0.7''")


sca = ax[-1].scatter(vdf['z'][dualmask],vdf['rp'][dualmask], cmap="magma",c=quant[dualmask])
cax = fig.add_axes([0.99, 0.125, 0.03, 0.25])
cbar = fig.colorbar(sca,cax=cax)
cbar.set_label("Log(L$_{\\rm{457 W, total}}$) [erg/s]")
ax[-1].set_xlabel("Redshift")
ax[-1].set_ylabel("Sep(kpc)")
ax[-1].legend();


fig.tight_layout();


### observed sep hist

In [ ]:
bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
# create masks
positive_sep = bigmac['Sep(kpc)'] > 0 
dual_sep = (bigmac['Sep(kpc)']>0.1 ) #& (bigmac['Sep(kpc)']<20)
confd = bigmac['ST1 Confidence Flag']>=0
radio = bigmac['Parsed Analysis Method'].str.contains("Radio")
imaging = bigmac['Parsed Analysis Method'].str.contains("Imaging")
# get papers with largest sample
pp = []
for i in bigmac['Paper(s)']:
    [pp.append(j) for j in i.split(" ; ")]
paper_dict = Counter(pp).most_common()

In [ ]:
def norm_hist(ax,quant,fcolor,ecolor,bin_arr,lbl):
    """normalize histogram sum count to 1 given some quantity (quant)
        args: edgecoloe, facecolor, bin array, opacity, flag for plotting horizontal hist"""
    count, bin = np.histogram(quant,bins=bin_arr)
    ax.bar(bin[:-1],count/np.sum(count),width = np.diff(bin),align='edge',facecolor=fcolor,edgecolor=ecolor,label=lbl)
    #ax.plot(bin[:-1],count/np.sum(count),c=ecolor)
    
def plot_sep_hist(npaper, upsep):
    fig,ax = plt.subplots(npaper//5,5,figsize=(12,3*npaper//5),sharex=True, sharey=True)
    ax = ax.ravel()
    upsepvar = np.max(vard)
    numbin = int(np.sqrt(len(vard)))
    bins_vard = np.linspace(np.min(vard),upsepvar,numbin)
    bins = np.diff(bins_vard)[0]*np.arange(upsep)+1
    cls = sns.color_palette("colorblind", 10)
    small_sep = bigmac['Sep(kpc)']< upsep
    for p,a in zip(paper_dict[:npaper+1],ax):
        mask = [np.isin(p[0],i.split(" ; ")) == True for i in bigmac['Paper(s)']]
        #plt.hist(bigmac[mask&positive_sep]['Sep(kpc)'],histtype="step",label=p[0])
        norm_hist(a,vard,"darkseagreen","darkseagreen",bins,'varstro')
        norm_hist(a,bigmac[mask & positive_sep & small_sep]['Sep(kpc)'],"none","blue",bins,p[0])
        a.legend()
    fig.tight_layout();
